# Gemini Code

In [ ]:
import numpy as np

# --- Helper Functions ---
def sigmoid(x):
    """Sigmoid activation function."""
    return 1 / (1 + np.exp(-x))

defmse_loss(y_true, y_pred):
    """Mean Squared Error loss."""
    return np.mean((y_true - y_pred)**2)

defcross_entropy_loss(y_true, y_pred):
    """Cross-entropy loss for binary data or logistic outputs.
    Assumes y_true are probabilities and y_pred are reconstructed probabilities.
    From the paper: [-sum_i p_i log(p_hat_i) - sum_i (1-p_i)log(1-p_hat_i)] [cite: 112]
    or [-sum_i p_i log(p_hat_i)] for multiclass cross-entropy [cite: 127]
    This example uses a common binary cross-entropy form.
    """
    epsilon = 1e-7 # to prevent log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

class RBM:
    """
    A Restricted Boltzmann Machine (RBM).
    This class implements the core logic for an RBM layer,
    which will be used for pretraining.
    """
    def __init__(self, n_visible, n_hidden, learning_rate=0.01, n_epochs=100, batch_size=32, k=1):
        """
        Initialize RBM parameters.
        n_visible: Number of visible units
        n_hidden: Number of hidden units
        learning_rate: Learning rate for weight updates
        n_epochs: Number of training epochs
        batch_size: Size of mini-batches for training
        k: Number of Gibbs sampling steps for Contrastive Divergence (CD-k)
        """
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.k = k # Number of steps in Contrastive Divergence

        # Initialize weights and biases
        # Weights are typically initialized with small random values
        self.W = np.random.randn(n_visible, n_hidden) * 0.1
        self.h_bias = np.zeros(n_hidden) # Hidden layer biases [cite: 79]
        self.v_bias = np.zeros(n_visible) # Visible layer biases [cite: 79]

    def _sample_h_given_v(self, v):
        """
        Sample hidden layer activations given visible layer states.
        P(h_j=1|v) = sigmoid(b_j + sum_i v_i * w_ij) [cite: 84]
        v: visible layer states (batch_size, n_visible)
        Returns:
            h_prob: hidden layer activation probabilities
            h_sample: sampled binary hidden layer states
        """
        h_prob = sigmoid(np.dot(v, self.W) + self.h_bias)
        h_sample = (h_prob > np.random.rand(h_prob.shape[0], h_prob.shape[1])).astype(np.float32)
        return h_prob, h_sample

    def _sample_v_given_h(self, h):
        """
        Sample visible layer activations (reconstruction) given hidden layer states.
        P(v_i=1|h) = sigmoid(b_i + sum_j h_j * w_ij) [cite: 85]
        (For Gaussian visible units, this would sample from a Gaussian distribution [cite: 104])
        h: hidden layer states (batch_size, n_hidden)
        Returns:
            v_prob: visible layer activation probabilities (reconstruction)
            v_sample: sampled binary visible layer states
        """
        v_prob = sigmoid(np.dot(h, self.W.T) + self.v_bias)
        v_sample = (v_prob > np.random.rand(v_prob.shape[0], v_prob.shape[1])).astype(np.float32)
        return v_prob, v_sample

    def contrastive_divergence(self, v_batch):
        """
        Perform one step of Contrastive Divergence (CD-k).
        v_batch: A mini-batch of visible data (batch_size, n_visible)
        """
        # Positive phase: data-driven
        # <v_i h_j>_data [cite: 89]
        h_prob_orig, h_sample_orig = self._sample_h_given_v(v_batch)
        
        # Negative phase: reconstruction-driven (Gibbs sampling)
        # <v_i h_j>_recon [cite: 89]
        v_sample_recon = None
        h_sample_recon = h_sample_orig
        for step in range(self.k):
            v_prob_recon, v_sample_recon = self._sample_v_given_h(h_sample_recon)
            h_prob_recon, h_sample_recon = self._sample_h_given_v(v_sample_recon)

        # Calculate gradients
        # delta_w_ij = epsilon * (<v_i h_j>_data - <v_i h_j>_recon) [cite: 89]
        positive_grad = np.dot(v_batch.T, h_prob_orig) # Using probabilities for smoother gradient
        negative_grad = np.dot(v_prob_recon.T, h_prob_recon) # Using probabilities for smoother gradient

        # Update weights and biases
        self.W += self.learning_rate * (positive_grad - negative_grad) / v_batch.shape[0]
        self.h_bias += self.learning_rate * np.mean(h_prob_orig - h_prob_recon, axis=0)
        self.v_bias += self.learning_rate * np.mean(v_batch - v_prob_recon, axis=0) # v_batch for original, v_prob_recon for reconstruction

        # Reconstruction error for monitoring
        error = np.mean((v_batch - v_prob_recon)**2)
        return error

    def train(self, data):
        """
        Train the RBM.
        data: Training data (n_samples, n_visible)
        """
        print(f"Training RBM: {self.n_visible} -> {self.n_hidden}")
        num_batches = int(np.ceil(data.shape[0] / self.batch_size))
        for epoch in range(self.n_epochs):
            epoch_error = 0
            # Shuffle data
            np.random.shuffle(data)
            for i in range(num_batches):
                batch_data = data[i*self.batch_size:(i+1)*self.batch_size]
                if batch_data.shape[0] == 0:
                    continue
                error = self.contrastive_divergence(batch_data)
                epoch_error += error
            
            print(f"  Epoch {epoch+1}/{self.n_epochs}, Reconstruction Error: {epoch_error/num_batches:.4f}")
        print(f"RBM training complete for {self.n_visible} -> {self.n_hidden}.\n")

    def transform(self, data):
        """
        Transform data by computing hidden layer activations.
        This is used as input for the next RBM in the stack. [cite: 70]
        data: Input data (n_samples, n_visible)
        Returns:
            hidden_activations: Probabilities of hidden unit activations
        """
        h_prob, _ = self._sample_h_given_v(data)
        # "While training higher level RBMs, the visible units
        # were set to the activation probabilities of the
        # hidden units in the previous RBM" [cite: 106]
        return h_prob


class DeepAutoencoder:
    """
    A Deep Autoencoder network.
    Can be initialized with pretrained RBM weights.
    """
    def __init__(self, layer_sizes, learning_rate=0.01, n_epochs_finetune=50, batch_size=32, use_cross_entropy_loss=False):
        """
        Initialize Deep Autoencoder.
        layer_sizes: List of integers specifying the number of neurons in each layer,
                     e.g., [784, 400, 200, 50] means 784 input, 2 hidden layers, 50 code layer.
                     The decoder will be symmetric.
        learning_rate: Learning rate for fine-tuning.
        n_epochs_finetune: Number of epochs for fine-tuning.
        batch_size: Mini-batch size for fine-tuning.
        use_cross_entropy_loss: Boolean, if True use cross-entropy, else MSE.
                                The paper mentions cross-entropy for logistic outputs. [cite: 112]
        """
        self.layer_sizes = layer_sizes
        self.n_layers = len(layer_sizes)
        self.learning_rate = learning_rate
        self.n_epochs_finetune = n_epochs_finetune
        self.batch_size = batch_size
        self.use_cross_entropy_loss = use_cross_entropy_loss

        self.encoder_weights = [] # List to store weights for encoder layers W1, W2, ...
        self.encoder_biases = []  # List to store biases for encoder layers
        self.decoder_weights = [] # List to store weights for decoder layers W_L^T, ... W1^T
        self.decoder_biases = []  # List to store biases for decoder layers

        # Note: For simplicity, this example uses sigmoid for all hidden layers.
        # The paper mentions code layers can be linear. [cite: 114, 122, 128]
        # And input/output layers might be linear with Gaussian noise or logistic. [cite: 104, 112, 105]

    def initialize_weights_from_rbms(self, rbm_stack):
        """
        Initialize encoder and decoder weights and biases from a stack of trained RBMs.
        This is the "unrolling" phase. [cite: 71]
        rbm_stack: A list of trained RBM objects.
        """
        print("Unrolling RBM weights into Deep Autoencoder...")
        if len(rbm_stack) != (self.n_layers - 1):
            raise ValueError("Number of RBMs must match number of autoencoder encoding layers.")

        for i in range(self.n_layers - 1): # For each encoding layer
            rbm = rbm_stack[i]
            self.encoder_weights.append(np.copy(rbm.W)) # W_i
            self.encoder_biases.append(np.copy(rbm.h_bias)) # bias for hidden layer of RBM_i

            # Decoder weights are transposes of encoder weights initially [cite: 102]
            # Decoder biases are the visible biases of the corresponding RBM
            self.decoder_weights.insert(0, np.copy(rbm.W.T)) # W_i^T
            self.decoder_biases.insert(0, np.copy(rbm.v_bias)) # bias for visible layer of RBM_i
        print("Autoencoder weights initialized from RBMs.\n")
        
    def _initialize_random_weights(self):
        """Initialize weights randomly if not pretraining."""
        print("Initializing Autoencoder weights randomly...")
        for i in range(self.n_layers - 1):
            # Encoder
            w_enc = np.random.randn(self.layer_sizes[i], self.layer_sizes[i+1]) * 0.1
            b_enc = np.zeros(self.layer_sizes[i+1])
            self.encoder_weights.append(w_enc)
            self.encoder_biases.append(b_enc)

            # Decoder (symmetric structure)
            w_dec = np.random.randn(self.layer_sizes[i+1], self.layer_sizes[i]) * 0.1
            b_dec = np.zeros(self.layer_sizes[i])
            self.decoder_weights.insert(0, w_dec) # Add to beginning for reverse order
            self.decoder_biases.insert(0, b_dec)
        print("Autoencoder weights initialized randomly.\n")


    def encode(self, x):
        """
        Encoder pass: data -> code
        x: input data (batch_size, n_features)
        Returns:
            activations: list of activations at each encoder layer (including code layer)
        """
        a = x
        activations = [a]
        for i in range(len(self.encoder_weights)):
            # The paper mentions the code layer can be linear [cite: 114]
            # For simplicity, we use sigmoid for all layers here.
            # A more complete implementation would allow specifying activation per layer.
            z = np.dot(a, self.encoder_weights[i]) + self.encoder_biases[i]
            a = sigmoid(z)
            activations.append(a)
        return activations # Last element is the code

    def decode(self, code_activations_list):
        """
        Decoder pass: code -> reconstruction
        code_activations_list: list of activations from the encoder pass,
                               needed for backpropagation. The actual code is the last element.
        Returns:
            reconstruction: reconstructed data
            decoder_activations: list of activations at each decoder layer (for backprop)
        """
        a = code_activations_list[-1] # This is the code layer output
        decoder_activations = [a] # Start with code layer activation

        for i in range(len(self.decoder_weights)):
            z = np.dot(a, self.decoder_weights[i]) + self.decoder_biases[i]
            # Output layer activation might differ (e.g. linear for PCA-like, or sigmoid for [0,1] data [cite: 112])
            # Assuming sigmoid for reconstruction to match input if it was [0,1]
            if i == len(self.decoder_weights) - 1: # Output layer
                 if self.use_cross_entropy_loss: # implies logistic output units [cite: 112]
                    a = sigmoid(z)
                 else: # Assuming linear output for MSE, or if data isn't strictly [0,1]
                    a = z # Or sigmoid(z) if input data was normalized to [0,1] and MSE is used
            else: # Hidden layers of decoder
                a = sigmoid(z)
            decoder_activations.append(a)
        return decoder_activations # Last element is the final reconstruction

    def forward_pass(self, x):
        """Full forward pass through encoder and decoder."""
        encoder_activations = self.encode(x)
        decoder_activations_including_code = self.decode(encoder_activations)
        return encoder_activations, decoder_activations_including_code

    def backpropagate(self, x_batch, encoder_activations, decoder_activations):
        """
        Perform backpropagation and update weights.
        "The required gradients are easily obtained by using the chain
         rule to backpropagate error derivatives first through the
         decoder network and then through the encoder network" [cite: 61]
        
        x_batch: Original input data for the batch
        encoder_activations: List of activations from encoder (input, h1, h2, ..., code)
        decoder_activations: List of activations from decoder (code, h_L-1_dec, ..., output)
        """
        
        # --- Decoder Weight Updates ---
        # Error at the output layer
        # The paper implies derivative of cross-entropy for logistic units, or MSE.
        # For cross-entropy with sigmoid output: error_delta = (reconstruction - true)
        # For MSE with linear output: error_delta = (reconstruction - true) * derivative_of_linear (which is 1)
        # For MSE with sigmoid output: error_delta = (reconstruction - true) * reconstruction * (1 - reconstruction)
        
        reconstruction = decoder_activations[-1]
        if self.use_cross_entropy_loss: # Assumes sigmoid output
            error_delta = (reconstruction - x_batch) # Derivative of CE with sigmoid
        else: # Assuming MSE
            if np.array_equal(reconstruction, sigmoid(np.dot(decoder_activations[-2], self.decoder_weights[-1]) + self.decoder_biases[-1])): # if output is sigmoid
                 error_delta = (reconstruction - x_batch) * reconstruction * (1 - reconstruction)
            else: # if output is linear
                 error_delta = (reconstruction - x_batch)

        # Iterate backwards through decoder layers
        decoder_weight_grads = []
        decoder_bias_grads = []

        num_decoder_hidden_layers = len(self.decoder_weights)
        for i in range(num_decoder_hidden_layers -1, -1, -1):
            # current layer output a_k (decoder_activations[i+1]), prev layer output a_j (decoder_activations[i])
            # Note: decoder_activations[0] is the code layer output.
            # So, decoder_activations[i] is input to weights self.decoder_weights[i]
            # and decoder_activations[i+1] is output of that layer.
            
            layer_input_activation = decoder_activations[i] # e.g. code for first iteration (i=0)
            
            # Gradient for weights
            dw = np.dot(layer_input_activation.T, error_delta)
            db = np.sum(error_delta, axis=0)
            decoder_weight_grads.insert(0, dw)
            decoder_bias_grads.insert(0, db)
            
            if i > 0: # Don't need to propagate error back from the first decoder layer (input to it is code)
                # Propagate error to the previous layer (current layer's input)
                error_delta = np.dot(error_delta, self.decoder_weights[i].T)
                # Multiply by derivative of activation function of layer_input_activation
                # (which is decoder_activations[i], the output of the *previous* actual layer,
                # or the input to the *current* weights W_dec[i])
                error_delta *= layer_input_activation * (1 - layer_input_activation) # Assuming sigmoid

        # --- Encoder Weight Updates ---
        # The error_delta now is at the output of the code layer (input to the first decoder layer)
        # It needs to be propagated back through the encoder.
        # The 'error_delta' from the last step of decoder backprop is delta for the code layer
        
        encoder_weight_grads = []
        encoder_bias_grads = []

        num_encoder_layers = len(self.encoder_weights) # e.g. 3 layers for 784-400-200-50
        # encoder_activations: [x, a_enc1, a_enc2, code_output]
        # So, encoder_activations[i] is input to self.encoder_weights[i]
        # and encoder_activations[i+1] is output.

        for i in range(num_encoder_layers - 1, -1, -1):
            layer_input_activation = encoder_activations[i] # e.g. x for first iteration
            layer_output_activation = encoder_activations[i+1] # e.g. a_enc1 for first iteration
            
            # Gradient for weights
            dw = np.dot(layer_input_activation.T, error_delta)
            db = np.sum(error_delta, axis=0)
            encoder_weight_grads.insert(0, dw)
            encoder_bias_grads.insert(0, db)
            
            if i > 0: # Don't propagate error back from input layer
                # Propagate error to the previous layer
                error_delta = np.dot(error_delta, self.encoder_weights[i].T)
                # Multiply by derivative of activation function of layer_input_activation
                # (which is encoder_activations[i], the output of the *previous* layer in encoder)
                error_delta *= layer_input_activation * (1 - layer_input_activation) # Assuming sigmoid

        # Apply updates (Gradient Descent)
        batch_size = x_batch.shape[0]
        for i in range(len(self.encoder_weights)):
            self.encoder_weights[i] -= self.learning_rate * encoder_weight_grads[i] / batch_size
            self.encoder_biases[i] -= self.learning_rate * encoder_bias_grads[i] / batch_size
        
        for i in range(len(self.decoder_weights)):
            self.decoder_weights[i] -= self.learning_rate * decoder_weight_grads[i] / batch_size
            self.decoder_biases[i] -= self.learning_rate * decoder_bias_grads[i] / batch_size


    def fine_tune(self, data):
        """
        Fine-tune the autoencoder using backpropagation. [cite: 71, 103]
        data: Training data (n_samples, n_features)
        """
        if not self.encoder_weights: # If not pretrained
            self._initialize_random_weights()

        print(f"Fine-tuning Deep Autoencoder: {'-'.join(map(str, self.layer_sizes))}")
        num_batches = int(np.ceil(data.shape[0] / self.batch_size))

        for epoch in range(self.n_epochs_finetune):
            epoch_loss = 0
            np.random.shuffle(data) # Shuffle data each epoch
            for i in range(num_batches):
                batch_data = data[i*self.batch_size:(i+1)*self.batch_size]
                if batch_data.shape[0] == 0:
                    continue

                # Forward pass
                encoder_activations, decoder_activations_full = self.forward_pass(batch_data)
                reconstruction = decoder_activations_full[-1]

                # Calculate loss
                if self.use_cross_entropy_loss:
                    loss = cross_entropy_loss(batch_data, reconstruction)
                else:
                    loss = mse_loss(batch_data, reconstruction)
                epoch_loss += loss

                # Backward pass (Backpropagation)
                self.backpropagate(batch_data, encoder_activations, decoder_activations_full)
            
            print(f"  Epoch {epoch+1}/{self.n_epochs_finetune}, Fine-tuning Loss: {epoch_loss/num_batches:.6f}")
        print("Autoencoder fine-tuning complete.\n")

    def get_code(self, data):
        """Get the low-dimensional code for the input data."""
        encoder_activations = self.encode(data)
        return encoder_activations[-1] # The last activation in the encoder is the code

    def reconstruct(self, data):
        """Reconstruct data using the autoencoder."""
        _, decoder_activations = self.forward_pass(data)
        return decoder_activations[-1]


# --- Main Execution ---
if __name__ == '__main__':
    # --- 1. Configuration ---
    # Example: For MNIST-like data (28x28 images = 784 features)
    # Autoencoder structure: 784 -> 400 -> 200 -> 50 (code layer)
    input_dim = 784
    # Layer sizes for the autoencoder (input -> h1 -> h2 -> code)
    # The paper mentions structures like (28*28)-400-200-100-50-25-6 [cite: 113]
    # or 784-1000-500-250-30 [cite: 120]
    autoencoder_layer_sizes = [input_dim, 400, 200, 50] 
    
    # RBM layers will be:
    # RBM1: input_dim -> 400
    # RBM2: 400 -> 200
    # RBM3: 200 -> 50 (This RBM learns features that will form the code layer)
    rbm_hidden_sizes = autoencoder_layer_sizes[1:] # [400, 200, 50]

    # Create some dummy data (e.g., random binary data for simplicity)
    # In practice, this would be your actual dataset (e.g., MNIST images)
    # Values should be in [0,1] if using sigmoid units and cross-entropy [cite: 112]
    print("Generating dummy data...")
    num_samples = 1000
    train_data = np.random.rand(num_samples, input_dim).astype(np.float32)
    # train_data = (train_data > 0.5).astype(np.float32) # For binary data
    print(f"Dummy data generated: {train_data.shape}\n")

    # --- 2. Pretraining Phase ---
    # "Pretraining consists of learning a stack of restricted Boltzmann machines (RBMs)" [cite: 69]
    print("--- Starting RBM Pretraining Phase ---")
    rbm_stack = []
    current_input_data = train_data
    current_input_size = input_dim

    for i, hidden_size in enumerate(rbm_hidden_sizes):
        print(f"Training RBM {i+1} ({current_input_size} -> {hidden_size})...")
        # RBM parameters can be tuned
        rbm = RBM(n_visible=current_input_size, 
                  n_hidden=hidden_size, 
                  learning_rate=0.05, 
                  n_epochs=10, # Short epochs for demo
                  batch_size=64,
                  k=1) # CD-1
        rbm.train(current_input_data)
        rbm_stack.append(rbm)
        
        # Get the activations of the hidden layer to use as input for the next RBM [cite: 70]
        print(f"Transforming data with RBM {i+1} to get input for next RBM...")
        current_input_data = rbm.transform(current_input_data)
        current_input_size = hidden_size
        print(f"New input data shape for next RBM: {current_input_data.shape}\n")
    print("--- RBM Pretraining Complete ---\n")

    # --- 3. Unrolling Phase & Autoencoder Initialization ---
    # "After the pretraining, the RBMs are "unrolled" to create a deep autoencoder" [cite: 71]
    print("--- Initializing Deep Autoencoder and Unrolling Weights ---")
    deep_ae = DeepAutoencoder(
        layer_sizes=autoencoder_layer_sizes,
        learning_rate=0.1, # Learning rate for fine-tuning
        n_epochs_finetune=20, # Short epochs for demo
        batch_size=64,
        # The paper uses cross-entropy for images with logistic output units [cite: 112]
        use_cross_entropy_loss=True # Set based on data and output units
    )
    deep_ae.initialize_weights_from_rbms(rbm_stack)
    
    # --- Alternative: Autoencoder without pretraining (for comparison) ---
    # print("--- Initializing Deep Autoencoder RANDOMLY (no pretraining) ---")
    # deep_ae_random = DeepAutoencoder(
    #     layer_sizes=autoencoder_layer_sizes,
    #     learning_rate=0.1,
    #     n_epochs_finetune=20,
    #     batch_size=64,
    #     use_cross_entropy_loss=True
    # )
    # # deep_ae_random._initialize_random_weights() # Called internally by fine_tune if not pretrained

    # --- 4. Fine-tuning Phase ---
    # "...which is then fine-tuned using backpropagation of error derivatives." [cite: 71]
    print("--- Starting Autoencoder Fine-tuning Phase (with pretrained weights) ---")
    deep_ae.fine_tune(train_data) # Use original full-dimensional data for fine-tuning

    # print("--- Starting Autoencoder Fine-tuning Phase (with random weights) ---")
    # deep_ae_random.fine_tune(train_data)
    # The paper notes: "With large initial weights, autoencoders typically find poor local minima;
    # with small initial weights, the gradients in the early layers are tiny...
    # If the initial weights are close to a good solution, gradient descent works well" [cite: 63, 64, 65]
    # And "Without pretraining, the very deep autoencoder always reconstructs the average of the
    # training data, even after prolonged fine-tuning (8)." (for a very deep example) [cite: 117]

    # --- 5. Using the Autoencoder ---
    print("--- Using the Trained Autoencoder ---")
    # Get low-dimensional codes
    sample_data_for_coding = train_data[:5]
    codes = deep_ae.get_code(sample_data_for_coding)
    print(f"Generated codes for 5 samples (shape: {codes.shape}):\n{codes}\n")

    # Reconstruct data
    reconstructions = deep_ae.reconstruct(sample_data_for_coding)
    print(f"Reconstructed 5 samples (shape: {reconstructions.shape}):\n{reconstructions}\n")
    
    original_vs_reconstructed_loss = cross_entropy_loss(sample_data_for_coding, reconstructions) if deep_ae.use_cross_entropy_loss else mse_loss(sample_data_for_coding, reconstructions)
    print(f"Final reconstruction loss on 5 samples: {original_vs_reconstructed_loss:.6f}")